In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 1. Collect a dataset of at least three landmarks. Your dataset should include at least 100 images of each in train, 50 in validation, and 25 in test (using more images is fine). You can randomly shuffle your dataset to create these splits.

3 labels: Central Park, Empire State Building, Statue of Liberty

I scraped approximately 100 images each from Google. 

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import os

# The code in this notebook should work identically between TF v1 and v2
import tensorflow as tf
import zipfile

from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, MobileNet

In [28]:
# Images will be resized to(TARGET_SHAPE, TARGET_SHAPE) as they're read off disk.
TARGET_SHAPE = 150 
BATCH_SIZE = 5

train_image_generator = ImageDataGenerator(rescale=1./255)
val_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(directory="/content/gdrive/My Drive/img/train", 
                                                     shuffle=True, # Best practice: shuffle the training data
                                                     target_size=(TARGET_SHAPE, TARGET_SHAPE),
                                                     class_mode='categorical')

val_data_gen = val_image_generator.flow_from_directory(directory="/content/gdrive/My Drive/img/val", 
                                                     shuffle=True, # Best practice: shuffle the training data
                                                     target_size=(TARGET_SHAPE, TARGET_SHAPE),
                                                     class_mode='categorical')

Found 228 images belonging to 3 classes.
Found 59 images belonging to 3 classes.


## 3 Next, how small of a model (in terms of the number of parameters) can you write to classify these images reasonably well? 

I tried 4 sets of convolution and maxpooling layers in the beginning, which gave me the validation accuracy of about 80%. Then I made the model even smaller with two sets of convolution and maxpooling layers, which resulted in ~90% validation accruacy. Then I tried one set, which yielded less than 40% accruacy. The smallest model I could achieve was two sets of convolution and maxpooling layers. I belive with too many sets of layers,the small nubmer of data I have causes overfitting, which gave me the lower accruacy in the first place .

In [21]:
EPOCHS = 7

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(233 / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(60 / float(BATCH_SIZE)))
)

Epoch 1/7
1/2 [==============>...............] - ETA: 9s - loss: 0.9241 - acc: 0.7500

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:885: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


7/7 [==============================] - 77s 11s/step - loss: 1.0901 - acc: 0.4144 - val_loss: 0.9376 - val_acc: 0.6441
Epoch 2/7
7/7 [==============================] - 19s 3s/step - loss: 0.8503 - acc: 0.6216 - val_loss: 0.7277 - val_acc: 0.5763
Epoch 3/7
7/7 [==============================] - 22s 3s/step - loss: 0.8125 - acc: 0.6036 - val_loss: 0.6622 - val_acc: 0.6949
Epoch 4/7
7/7 [==============================] - 22s 3s/step - loss: 0.7113 - acc: 0.6667 - val_loss: 0.6930 - val_acc: 0.7119
Epoch 5/7
7/7 [==============================] - 22s 3s/step - loss: 0.6618 - acc: 0.7072 - val_loss: 0.6129 - val_acc: 0.6610
Epoch 6/7
7/7 [==============================] - 23s 3s/step - loss: 0.5358 - acc: 0.7973 - val_loss: 0.4313 - val_acc: 0.8305
Epoch 7/7
7/7 [==============================] - 22s 3s/step - loss: 0.4507 - acc: 0.8153 - val_loss: 0.5416 - val_acc: 0.7966


In [29]:
EPOCHS = 7

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(233 / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(60 / float(BATCH_SIZE)))
)

Epoch 1/7
7/8 [=========================>....] - ETA: 2s - loss: 2.0362 - acc: 0.3469

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:885: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


8/8 [==============================] - 21s 3s/step - loss: 1.8689 - acc: 0.3509 - val_loss: 0.9917 - val_acc: 0.3559
Epoch 2/7
8/8 [==============================] - 19s 2s/step - loss: 0.8892 - acc: 0.5746 - val_loss: 0.6990 - val_acc: 0.7119
Epoch 3/7
8/8 [==============================] - 19s 2s/step - loss: 0.5906 - acc: 0.7412 - val_loss: 0.6519 - val_acc: 0.6780
Epoch 4/7
8/8 [==============================] - 19s 2s/step - loss: 0.4523 - acc: 0.7939 - val_loss: 0.5016 - val_acc: 0.8136
Epoch 5/7
8/8 [==============================] - 19s 2s/step - loss: 0.3174 - acc: 0.8596 - val_loss: 0.4960 - val_acc: 0.8136
Epoch 6/7
8/8 [==============================] - 19s 2s/step - loss: 0.1998 - acc: 0.9298 - val_loss: 0.3590 - val_acc: 0.8983
Epoch 7/7
8/8 [==============================] - 19s 2s/step - loss: 0.1265 - acc: 0.9605 - val_loss: 0.2183 - val_acc: 0.8983


In [30]:
EPOCHS = 7

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(233 / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(60 / float(BATCH_SIZE)))
)

Epoch 1/7
7/8 [=========================>....] - ETA: 1s - loss: 9.6559 - acc: 0.3163

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:885: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


8/8 [==============================] - 16s 2s/step - loss: 9.9310 - acc: 0.3070 - val_loss: 10.4283 - val_acc: 0.3390
Epoch 2/7
8/8 [==============================] - 16s 2s/step - loss: 10.6119 - acc: 0.3246 - val_loss: 10.5682 - val_acc: 0.3390
Epoch 3/7
8/8 [==============================] - 16s 2s/step - loss: 10.6119 - acc: 0.3246 - val_loss: 10.8014 - val_acc: 0.3390
Epoch 4/7
8/8 [==============================] - 15s 2s/step - loss: 11.0036 - acc: 0.3246 - val_loss: 10.5682 - val_acc: 0.3390
Epoch 5/7
8/8 [==============================] - 16s 2s/step - loss: 11.0036 - acc: 0.3246 - val_loss: 10.7081 - val_acc: 0.3390
Epoch 6/7
8/8 [==============================] - 15s 2s/step - loss: 10.6119 - acc: 0.3246 - val_loss: 10.6614 - val_acc: 0.3390
Epoch 7/7
8/8 [==============================] - 15s 2s/step - loss: 11.0036 - acc: 0.3246 - val_loss: 10.6148 - val_acc: 0.3390


## 2 Write a model to classify your dataset using transfer learning. Run an experiment and report your results. What do you find?

After setting the VGG16 convolutional base not trainable, I added a simple 2-layer model of 64-node dense layer and 3-node dense layer. I trained with an adam optimizer and cross entropy loss function. I found that my model is doing pretty well, because validation accruacy is about 95%, and it started with a pretty high validation accuracy of 85%. My reasoning of the high validation accuracy is because from the transfer learning, the model already learned basics of images such as edges, shapes, so it didn't take long for models to learn about my dataset. 

In [24]:
conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(150, 150, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

conv_base.trainable = False

model.compile(
  optimizer=tf.train.AdamOptimizer(),
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(233 / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(60 / float(BATCH_SIZE)))
)

58892288/58889256 [==============================] - 1s 0us/step
Epoch 1/7
6/7 [========================>.....] - ETA: 8s - loss: 1.0665 - acc: 0.5895 

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:885: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


7/7 [==============================] - 81s 12s/step - loss: 1.0206 - acc: 0.6081 - val_loss: 0.4504 - val_acc: 0.8475
Epoch 2/7
7/7 [==============================] - 75s 11s/step - loss: 0.3321 - acc: 0.9144 - val_loss: 0.2166 - val_acc: 0.9322
Epoch 3/7
7/7 [==============================] - 74s 11s/step - loss: 0.1984 - acc: 0.9279 - val_loss: 0.1833 - val_acc: 0.9322
Epoch 4/7
7/7 [==============================] - 74s 11s/step - loss: 0.1053 - acc: 0.9640 - val_loss: 0.1157 - val_acc: 0.9492
Epoch 5/7
7/7 [==============================] - 74s 11s/step - loss: 0.0609 - acc: 0.9865 - val_loss: 0.1549 - val_acc: 0.9153
Epoch 6/7
7/7 [==============================] - 74s 11s/step - loss: 0.0417 - acc: 0.9955 - val_loss: 0.1282 - val_acc: 0.9492
Epoch 7/7
7/7 [==============================] - 74s 11s/step - loss: 0.0303 - acc: 1.0000 - val_loss: 0.0947 - val_acc: 0.9492


## 3 Explore the available pretrained ​models​, and see if any are suitable. Run an experiment and report your results.

I tried VGG16 and MobileNet for pretrained models. MobileNet's validation accuracy is less than 70%, but VGG16's was 95%. I believe the reason is that MobileNet is built for mobile devices so it is mindful of the restricted resources. However, VGG16 trained on a less resource-restricted environment. 

In [33]:
# Images will be resized to(TARGET_SHAPE, TARGET_SHAPE) as they're read off disk.
TARGET_SHAPE = 224 
BATCH_SIZE = 5

train_image_generator = ImageDataGenerator(rescale=1./255)
val_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(directory="/content/gdrive/My Drive/img/train", 
                                                     shuffle=True, # Best practice: shuffle the training data
                                                     target_size=(TARGET_SHAPE, TARGET_SHAPE),
                                                     class_mode='categorical')

val_data_gen = val_image_generator.flow_from_directory(directory="/content/gdrive/My Drive/img/val", 
                                                     shuffle=True, # Best practice: shuffle the training data
                                                     target_size=(TARGET_SHAPE, TARGET_SHAPE),
                                                     class_mode='categorical')

conv_base = MobileNet(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

conv_base.trainable = False

model.compile(
  optimizer=tf.train.AdamOptimizer(),
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(233 / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(60 / float(BATCH_SIZE)))
)

Found 228 images belonging to 3 classes.
Found 59 images belonging to 3 classes.
17227776/17225924 [==============================] - 0s 0us/step
Epoch 1/7
7/8 [=========================>....] - ETA: 3s - loss: 7.0896 - acc: 0.5204

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:885: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


8/8 [==============================] - 41s 5s/step - loss: 6.7448 - acc: 0.5439 - val_loss: 7.1586 - val_acc: 0.5085
Epoch 2/7
8/8 [==============================] - 36s 5s/step - loss: 7.0424 - acc: 0.5746 - val_loss: 5.5456 - val_acc: 0.6441
Epoch 3/7
8/8 [==============================] - 38s 5s/step - loss: 5.4912 - acc: 0.6360 - val_loss: 6.2977 - val_acc: 0.6102
Epoch 4/7
8/8 [==============================] - 34s 4s/step - loss: 5.9196 - acc: 0.6447 - val_loss: 6.3987 - val_acc: 0.6102
Epoch 5/7
8/8 [==============================] - 39s 5s/step - loss: 5.4468 - acc: 0.6447 - val_loss: 5.8154 - val_acc: 0.6271
Epoch 6/7
8/8 [==============================] - 36s 4s/step - loss: 5.0116 - acc: 0.6491 - val_loss: 5.4459 - val_acc: 0.6441
Epoch 7/7
8/8 [==============================] - 37s 5s/step - loss: 6.3918 - acc: 0.6667 - val_loss: 5.3167 - val_acc: 0.6610
